In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
!pip install snscrape

SNscrape

In [34]:
stock = input('What stock are you interested in?: ')


What stock are you interested in?: PFE


In [35]:
!pip install yfinance
import yfinance as yf

data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = stock,

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "60d",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "30m",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Datetime,,,,,
2020-12-24 09:30:00-05:00,37.400002,37.570000,37.180000,37.302101,3708175
2020-12-24 10:00:00-05:00,37.310001,37.520000,37.262100,37.480000,2236715
2020-12-24 10:30:00-05:00,37.476501,37.490002,37.275002,37.389999,2009427
2020-12-24 11:00:00-05:00,37.389999,37.435001,37.320000,37.410000,1494704
2020-12-24 11:30:00-05:00,37.404999,37.410000,37.250000,37.264900,1331516
...,...,...,...,...,...
2021-03-23 13:30:00-04:00,35.625000,35.677200,35.599998,35.645000,1050785
2021-03-23 14:00:00-04:00,35.645000,35.650002,35.570000,35.570000,1233770
2021-03-23 14:30:00-04:00,35.575001,35.605000,35.520000,35.520000,1010529


In [36]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime
# from datetime import datetime, timedelta


start = min(data.index)
end = max(data.index)

def workdays(d, end, excluded=(6, 7)):
    days = []
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d)
        d += datetime.timedelta(days=1)
    return days

total_days = workdays(datetime.datetime(start.year, start.month, start.day), datetime.datetime(end.year, end.month, end.day))
listofdates = []
for i in range (len(total_days)):
  date_time = total_days[i] + datetime.timedelta(minutes=30)
  date_time = date_time + datetime.timedelta(hours=15)
  end_time = date_time.strftime("%Y-%m-%d")  
  listofdates.append(end_time)

from datetime import datetime, timedelta
last = datetime.strptime(listofdates[-1] , '%Y-%m-%d')
last = last + timedelta(days=1)
last = last.strftime("%Y-%m-%d") 
listofdates.append(last)

# listofdates
tweets_df1 = pd.DataFrame()

for i in range(len(listofdates)):
  tweets_list = []
  if i == len(listofdates)-1:
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(str(stock) + ' lang:en since:' + str(listofdates[i]) + ' until:' + str(listofdates[i])).get_items()):
      tweets_list.append([tweet.date, tweet.content])

  else:
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(str(stock) + ' lang:en since:' + str(listofdates[i]) + ' until:' + str(listofdates[i+1])).get_items()):
      tweets_list.append([tweet.date, tweet.content])

  tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Text'])
  tweets_df1 = tweets_df1.append(tweets_df[::-1]) 
  

In [37]:
tweets_df1

,Datetime,Text
306,2020-12-24 00:03:01+00:00,S.Korea signs deals to secure COVID-19 vaccine...
305,2020-12-24 00:26:37+00:00,Major new 12/23\n$PFE\n$AAPL\n$AMZN\n$TSLA\n$G...
304,2020-12-24 00:27:18+00:00,Nikola is Officially a Nonstarter 🔋⚡️🛰🚗💨 $QS $...
303,2020-12-24 00:28:42+00:00,🚨Podcast alert:🚨\n\nThis week on Talking Stock...
302,2020-12-24 00:34:01+00:00,UPDATE 1-S.Korea signs deals to import Pfizer ...
...,...,...
4,2021-03-23 23:44:07+00:00,Back to back Steves? Nahhhh ima sit this one o...
3,2021-03-23 23:44:27+00:00,And to think I was actually gonna start compet...
2,2021-03-23 23:45:43+00:00,@ProfessorTimbo I can count on one hand how ma...
1,2021-03-23 23:47:55+00:00,I am finally continuing the (not so) long awai...


In [38]:
import pandas as pd
import numpy as np

In [39]:
!pip install emoji

In [40]:
import re
import emoji
tweets_df1['Text'] = tweets_df1['Text'].astype(str)

def cleaner(tweet):
    tweet = re.sub(r"(^|\W)\d+", "", tweet)# remove digits
    tweet = tweet.lower()
    if tweet[:2] == 'rt':
        tweet = re.sub(r'^.*?:', ':', tweet)
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ")
    tweet = tweet.replace("RT ", "")
    tweet = re.sub(r'[^\w\s]','',tweet) #remove punctuation
    return tweet
tweets_df1['processed_tweet'] = tweets_df1['Text'].map(lambda x: cleaner(x))

In [41]:
tweets_df1

,Datetime,Text,processed_tweet
306,2020-12-24 00:03:01+00:00,S.Korea signs deals to secure COVID-19 vaccine...,skorea signs deals to secure covid vaccine wit...
305,2020-12-24 00:26:37+00:00,Major new 12/23\n$PFE\n$AAPL\n$AMZN\n$TSLA\n$G...,major new pfe aapl amzn tsla goog googl bntx m...
304,2020-12-24 00:27:18+00:00,Nikola is Officially a Nonstarter 🔋⚡️🛰🚗💨 $QS $...,nikola is officially a nonstarter qs nkla tsl...
303,2020-12-24 00:28:42+00:00,🚨Podcast alert:🚨\n\nThis week on Talking Stock...,podcast alert this week on talking stocks eve...
302,2020-12-24 00:34:01+00:00,UPDATE 1-S.Korea signs deals to import Pfizer ...,updateskorea signs deals to import pfizer and ...
...,...,...,...
4,2021-03-23 23:44:07+00:00,Back to back Steves? Nahhhh ima sit this one o...,back to back steves nahhhh ima sit this one out
3,2021-03-23 23:44:27+00:00,And to think I was actually gonna start compet...,and to think i was actually gonna start compet...
2,2021-03-23 23:45:43+00:00,@ProfessorTimbo I can count on one hand how ma...,i can count on one hand how many times i met w...
1,2021-03-23 23:47:55+00:00,I am finally continuing the (not so) long awai...,i am finally continuing the not so long awaite...


In [42]:
# tweets_df1.to_csv('/content/drive/MyDrive/FA PROJECT 2021/LSTM/PFE/60days_tweets.csv', index=False)

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [44]:
prices = data.reset_index()
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 775 entries, 0 to 774
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype                           
---  ------    --------------  -----                           
 0   Datetime  775 non-null    datetime64[ns, America/New_York]
 1   Open      775 non-null    float64                         
 2   High      775 non-null    float64                         
 3   Low       775 non-null    float64                         
 4   Close     775 non-null    float64                         
 5   Volume    775 non-null    int64                           
dtypes: datetime64[ns, America/New_York](1), float64(4), int64(1)
memory usage: 36.5 KB


In [45]:
prices['Datetime'] = prices['Datetime'].astype(str)

In [46]:
#remove -5:00
prices['Datetime'] = prices['Datetime'].str.replace('-05:00',"")
prices['Datetime'] = prices['Datetime'].str.replace('-04:00',"")

In [51]:
prices = prices[(prices['Datetime']>= min(prices['Datetime'])) & (prices['Datetime']<=max(prices['Datetime']))]
prices

,Datetime,Open,High,Low,Close,Volume
0,2020-12-24 09:30:00,37.400002,37.570000,37.180000,37.302101,3708175
1,2020-12-24 10:00:00,37.310001,37.520000,37.262100,37.480000,2236715
2,2020-12-24 10:30:00,37.476501,37.490002,37.275002,37.389999,2009427
3,2020-12-24 11:00:00,37.389999,37.435001,37.320000,37.410000,1494704
4,2020-12-24 11:30:00,37.404999,37.410000,37.250000,37.264900,1331516
...,...,...,...,...,...,...
770,2021-03-23 13:30:00,35.625000,35.677200,35.599998,35.645000,1050785
771,2021-03-23 14:00:00,35.645000,35.650002,35.570000,35.570000,1233770
772,2021-03-23 14:30:00,35.575001,35.605000,35.520000,35.520000,1010529
773,2021-03-23 15:00:00,35.525002,35.540001,35.330002,35.360001,2224822


In [52]:
sentiment = tweets_df1

In [53]:
sentiment = sentiment.rename(columns={'Datetime': 'created_at'})

In [55]:
#remove +00:00
sentiment['created_at'] = sentiment['created_at'].astype(str)
sentiment['created_at'] = sentiment['created_at'].str.slice(0,19)
sentiment.head()

,created_at,Text,processed_tweet
306,2020-12-24 00:03:01,S.Korea signs deals to secure COVID-19 vaccine...,skorea signs deals to secure covid vaccine wit...
305,2020-12-24 00:26:37,Major new 12/23\n$PFE\n$AAPL\n$AMZN\n$TSLA\n$G...,major new pfe aapl amzn tsla goog googl bntx m...
304,2020-12-24 00:27:18,Nikola is Officially a Nonstarter 🔋⚡️🛰🚗💨 $QS $...,nikola is officially a nonstarter qs nkla tsl...
303,2020-12-24 00:28:42,🚨Podcast alert:🚨\n\nThis week on Talking Stock...,podcast alert this week on talking stocks eve...
302,2020-12-24 00:34:01,UPDATE 1-S.Korea signs deals to import Pfizer ...,updateskorea signs deals to import pfizer and ...


In [56]:
# convert datetime and created at to same datetime object
prices['Datetime'] = pd.to_datetime(prices['Datetime'])
sentiment['created_at'] = pd.to_datetime(sentiment['created_at'])

In [57]:
# only close column is needed
prices = prices.drop(columns=['Open','High','Low','Volume'])

In [58]:
sentiment = sentiment.rename(columns={'created_at': 'Datetime'})
sentiment

,Datetime,Text,processed_tweet
306,2020-12-24 00:03:01,S.Korea signs deals to secure COVID-19 vaccine...,skorea signs deals to secure covid vaccine wit...
305,2020-12-24 00:26:37,Major new 12/23\n$PFE\n$AAPL\n$AMZN\n$TSLA\n$G...,major new pfe aapl amzn tsla goog googl bntx m...
304,2020-12-24 00:27:18,Nikola is Officially a Nonstarter 🔋⚡️🛰🚗💨 $QS $...,nikola is officially a nonstarter qs nkla tsl...
303,2020-12-24 00:28:42,🚨Podcast alert:🚨\n\nThis week on Talking Stock...,podcast alert this week on talking stocks eve...
302,2020-12-24 00:34:01,UPDATE 1-S.Korea signs deals to import Pfizer ...,updateskorea signs deals to import pfizer and ...
...,...,...,...
4,2021-03-23 23:44:07,Back to back Steves? Nahhhh ima sit this one o...,back to back steves nahhhh ima sit this one out
3,2021-03-23 23:44:27,And to think I was actually gonna start compet...,and to think i was actually gonna start compet...
2,2021-03-23 23:45:43,@ProfessorTimbo I can count on one hand how ma...,i can count on one hand how many times i met w...
1,2021-03-23 23:47:55,I am finally continuing the (not so) long awai...,i am finally continuing the not so long awaite...


In [59]:
# drop na of sentiment
sentiment = sentiment.dropna()

In [60]:
df_merge = pd.merge_asof(sentiment,prices,left_on='Datetime',right_on="Datetime",direction='backward')
df_merge

,Datetime,Text,processed_tweet,Close
0,2020-12-24 00:03:01,S.Korea signs deals to secure COVID-19 vaccine...,skorea signs deals to secure covid vaccine wit...,NaN
1,2020-12-24 00:26:37,Major new 12/23\n$PFE\n$AAPL\n$AMZN\n$TSLA\n$G...,major new pfe aapl amzn tsla goog googl bntx m...,NaN
2,2020-12-24 00:27:18,Nikola is Officially a Nonstarter 🔋⚡️🛰🚗💨 $QS $...,nikola is officially a nonstarter qs nkla tsl...,NaN
3,2020-12-24 00:28:42,🚨Podcast alert:🚨\n\nThis week on Talking Stock...,podcast alert this week on talking stocks eve...,NaN
4,2020-12-24 00:34:01,UPDATE 1-S.Korea signs deals to import Pfizer ...,updateskorea signs deals to import pfizer and ...,NaN
...,...,...,...,...
24543,2021-03-23 23:44:07,Back to back Steves? Nahhhh ima sit this one o...,back to back steves nahhhh ima sit this one out,35.369999
24544,2021-03-23 23:44:27,And to think I was actually gonna start compet...,and to think i was actually gonna start compet...,35.369999
24545,2021-03-23 23:45:43,@ProfessorTimbo I can count on one hand how ma...,i can count on one hand how many times i met w...,35.369999
24546,2021-03-23 23:47:55,I am finally continuing the (not so) long awai...,i am finally continuing the not so long awaite...,35.369999


In [61]:
df_merge1 = df_merge[df_merge.Close.notnull()]
df_merge1

,Datetime,Text,processed_tweet,Close
77,2020-12-24 09:32:05,Sentiment heats up and the hysteria of retails...,sentiment heats up and the hysteria of retails...,37.302101
78,2020-12-24 09:36:54,"Fauci said he felt a 'little warm, a little FL...",fauci said he felt a little warm a little flus...,37.302101
79,2020-12-24 09:41:14,S&amp;P 500/VIX \nv. \nDownside/Upside Probabi...,samppvix v downsideupside probability ratio be...,37.302101
80,2020-12-24 09:47:20,Wattnow / propose son [PFE BOOK 2021] https://...,wattnow propose son pfe book,37.302101
81,2020-12-24 10:05:56,Nurses Celebrating Covid-19 Vaccines Battle So...,nurses celebrating covid vaccines battle socia...,37.480000
...,...,...,...,...
24543,2021-03-23 23:44:07,Back to back Steves? Nahhhh ima sit this one o...,back to back steves nahhhh ima sit this one out,35.369999
24544,2021-03-23 23:44:27,And to think I was actually gonna start compet...,and to think i was actually gonna start compet...,35.369999
24545,2021-03-23 23:45:43,@ProfessorTimbo I can count on one hand how ma...,i can count on one hand how many times i met w...,35.369999
24546,2021-03-23 23:47:55,I am finally continuing the (not so) long awai...,i am finally continuing the not so long awaite...,35.369999


In [63]:
import datetime

df_final = pd.DataFrame()

start = min(data.index)
end = max(data.index)

def workdays(d, end, excluded=(6, 7)):
    days = []
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d)
        d += datetime.timedelta(days=1)
    return days

total_days = workdays(datetime.datetime(start.year, start.month, start.day), datetime.datetime(end.year, end.month, end.day))
listofdates = []
for i in range (len(total_days)):
  date_time = total_days[i] + datetime.timedelta(minutes=30)
  date_time = date_time + datetime.timedelta(hours=15)
  end_time = date_time.strftime("%Y-%m-%d %H:%M:%S")

  date_time2 = total_days[i] + datetime.timedelta(hours=9)
  start_time = date_time2.strftime("%Y-%m-%d %H:%M:%S")
  
  listofdates.append([end_time, start_time])

# print(len(total_days))

for i in range(len(total_days)):
  df_final = df_final.append(df_merge1[(df_merge1['Datetime']<=listofdates[i][0]) & (df_merge1['Datetime']>=listofdates[i][1])])

In [64]:
df_final

,Datetime,Text,processed_tweet,Close
77,2020-12-24 09:32:05,Sentiment heats up and the hysteria of retails...,sentiment heats up and the hysteria of retails...,37.302101
78,2020-12-24 09:36:54,"Fauci said he felt a 'little warm, a little FL...",fauci said he felt a little warm a little flus...,37.302101
79,2020-12-24 09:41:14,S&amp;P 500/VIX \nv. \nDownside/Upside Probabi...,samppvix v downsideupside probability ratio be...,37.302101
80,2020-12-24 09:47:20,Wattnow / propose son [PFE BOOK 2021] https://...,wattnow propose son pfe book,37.302101
81,2020-12-24 10:05:56,Nurses Celebrating Covid-19 Vaccines Battle So...,nurses celebrating covid vaccines battle socia...,37.480000
...,...,...,...,...
24361,2021-03-23 15:24:17,Pfizer (PFE): Ultra-Cold Storage Of COVID-19 V...,pfizer pfe ultracold storage of covid vaccine ...,35.360001
24362,2021-03-23 15:26:33,@RobertLKruse @RNAiAnalyst Doesn't this open P...,doesnt this open pfe up to ip or technology li...,35.360001
24363,2021-03-23 15:27:09,Bad relations between #UnitedStates and #China...,bad relations between unitedstates and china m...,35.360001
24364,2021-03-23 15:28:47,Pfizer begins early-stage study of oral COVID-...,pfizer begins earlystage study of oral covid d...,35.360001


In [65]:
df_final = df_final.reset_index(drop=True)
df_final

,Datetime,Text,processed_tweet,Close
0,2020-12-24 09:32:05,Sentiment heats up and the hysteria of retails...,sentiment heats up and the hysteria of retails...,37.302101
1,2020-12-24 09:36:54,"Fauci said he felt a 'little warm, a little FL...",fauci said he felt a little warm a little flus...,37.302101
2,2020-12-24 09:41:14,S&amp;P 500/VIX \nv. \nDownside/Upside Probabi...,samppvix v downsideupside probability ratio be...,37.302101
3,2020-12-24 09:47:20,Wattnow / propose son [PFE BOOK 2021] https://...,wattnow propose son pfe book,37.302101
4,2020-12-24 10:05:56,Nurses Celebrating Covid-19 Vaccines Battle So...,nurses celebrating covid vaccines battle socia...,37.480000
...,...,...,...,...
6561,2021-03-23 15:24:17,Pfizer (PFE): Ultra-Cold Storage Of COVID-19 V...,pfizer pfe ultracold storage of covid vaccine ...,35.360001
6562,2021-03-23 15:26:33,@RobertLKruse @RNAiAnalyst Doesn't this open P...,doesnt this open pfe up to ip or technology li...,35.360001
6563,2021-03-23 15:27:09,Bad relations between #UnitedStates and #China...,bad relations between unitedstates and china m...,35.360001
6564,2021-03-23 15:28:47,Pfizer begins early-stage study of oral COVID-...,pfizer begins earlystage study of oral covid d...,35.360001


In [66]:
df_final["Comp"] = ''
df_final["Negative"] = ''
df_final["Neutral"] = ''
df_final["Positive"] = ''
df_final

,Datetime,Text,processed_tweet,Close,Comp,Negative,Neutral,Positive
0,2020-12-24 09:32:05,Sentiment heats up and the hysteria of retails...,sentiment heats up and the hysteria of retails...,37.302101,,,,
1,2020-12-24 09:36:54,"Fauci said he felt a 'little warm, a little FL...",fauci said he felt a little warm a little flus...,37.302101,,,,
2,2020-12-24 09:41:14,S&amp;P 500/VIX \nv. \nDownside/Upside Probabi...,samppvix v downsideupside probability ratio be...,37.302101,,,,
3,2020-12-24 09:47:20,Wattnow / propose son [PFE BOOK 2021] https://...,wattnow propose son pfe book,37.302101,,,,
4,2020-12-24 10:05:56,Nurses Celebrating Covid-19 Vaccines Battle So...,nurses celebrating covid vaccines battle socia...,37.480000,,,,
...,...,...,...,...,...,...,...,...
6561,2021-03-23 15:24:17,Pfizer (PFE): Ultra-Cold Storage Of COVID-19 V...,pfizer pfe ultracold storage of covid vaccine ...,35.360001,,,,
6562,2021-03-23 15:26:33,@RobertLKruse @RNAiAnalyst Doesn't this open P...,doesnt this open pfe up to ip or technology li...,35.360001,,,,
6563,2021-03-23 15:27:09,Bad relations between #UnitedStates and #China...,bad relations between unitedstates and china m...,35.360001,,,,
6564,2021-03-23 15:28:47,Pfizer begins early-stage study of oral COVID-...,pfizer begins earlystage study of oral covid d...,35.360001,,,,


In [67]:
df_final['processed_tweet'] = df_final['processed_tweet'].astype(str)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6566 entries, 0 to 6565
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Datetime         6566 non-null   datetime64[ns]
 1   Text             6566 non-null   object        
 2   processed_tweet  6566 non-null   object        
 3   Close            6566 non-null   float64       
 4   Comp             6566 non-null   object        
 5   Negative         6566 non-null   object        
 6   Neutral          6566 non-null   object        
 7   Positive         6566 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 410.5+ KB


In [68]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
df_final['Comp'] = [analyzer.polarity_scores(x)['compound'] for x in df_final['processed_tweet']]
df_final['Negative'] = [analyzer.polarity_scores(x)['neg'] for x in df_final['processed_tweet']]
df_final['Neutral'] = [analyzer.polarity_scores(x)['neu'] for x in df_final['processed_tweet']]
df_final['Positive'] = [analyzer.polarity_scores(x)['pos'] for x in df_final['processed_tweet']]

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [69]:
df_final = df_final.groupby(df_final.Datetime.dt.floor('30min')).mean()

In [70]:
df_final = df_final.drop(columns=['Negative','Neutral','Positive'])
df_final

,Close,Comp
Datetime,,
2020-12-24 09:30:00,37.302101,0.014600
2020-12-24 10:00:00,37.480000,0.062200
2020-12-24 10:30:00,37.389999,0.411700
2020-12-24 11:00:00,37.410000,0.157233
2020-12-24 11:30:00,37.264900,0.331650
...,...,...
2021-03-23 13:00:00,35.625000,0.199936
2021-03-23 13:30:00,35.645000,0.152037
2021-03-23 14:00:00,35.570000,0.273920


In [71]:
df_final.to_csv('/content/drive/MyDrive/FA PROJECT 2021/LSTM/PFE/final.csv')